# PROMPT & IMAGE AUGMENTATION

This is a temporary notebook in which we will generate the prompt for clip given the set of classes from Flowers102.

In [1]:
# Setup
# ! pip install -q -U google-genai
# ! pip install tqdm
# ! pip install openai_clip

import os
import torch
import json
import numpy as np
import torch.nn.functional as F

from tqdm import tqdm
from collections import defaultdict

  Using cached openai_clip-1.0.1-py3-none-any.whl
  Using cached ftfy-6.3.1-py3-none-any.whl.metadata (7.3 kB)
Using cached ftfy-6.3.1-py3-none-any.whl (44 kB)


**LIST of FLOWERS**
First we define the label name of all the 102 flower classes

In [2]:
CLASS_NAMES = ["pink primrose", "hard-leaved pocket orchid", "canterbury bells", "sweet pea", 
               "english marigold", "tiger lily", "moon orchid", "bird of paradise", 
               "monkshood", "globe thistle", "snapdragon", "colt's foot", "king protea", 
               "spear thistle", "yellow iris", "globe-flower", "purple coneflower", 
               "peruvian lily", "balloon flower", "giant white arum lily", "fire lily", 
               "pincushion flower", "fritillary", "red ginger", "grape hyacinth", "corn poppy", 
               "prince of wales feathers", "stemless gentian", "artichoke", "sweet william", 
               "carnation", "garden phlox", "love in the mist", "mexican aster", 
               "alpine sea holly", "ruby-lipped cattleya", "cape flower", "great masterwort", 
               "siam tulip", "lenten rose", "barbeton daisy", "daffodil", "sword lily", 
               "poinsettia", "bolero deep blue", "wallflower", "marigold", "buttercup", 
               "oxeye daisy", "common dandelion", "petunia", "wild pansy", "primula", 
               "sunflower", "pelargonium", "bishop of llandaff", "gaura", "geranium", 
               "orange dahlia", "pink-yellow dahlia?", "cautleya spicata", "japanese anemone", 
               "black-eyed susan", "silverbush", "californian poppy", "osteospermum", 
               "spring crocus", "bearded iris", "windflower", "tree poppy", "gazania", 
               "azalea", "water lily", "rose", "thorn apple", "morning glory", "passion flower",
               "lotus", "toad lily", "anthurium", "frangipani", "clematis", "hibiscus", 
               "columbine", "desert-rose", "tree mallow", "magnolia", "cyclamen", "watercress", 
               "canna lily", "hippeastrum", "bee balm", "ball moss", "foxglove", "bougainvillea", 
               "camellia", "mallow", "mexican petunia", "bromelia", "blanket flower", 
               "trumpet creeper", "blackberry lily"]

CLASS_NAMES_12 = ["pink primrose", "hard-leaved pocket orchid", "canterbury bells", "sweet pea", 
               "english marigold", "tiger lily", "moon orchid", "bird of paradise", 
               "monkshood", "globe thistle", "snapdragon", "colt's foot"]

### Prompt Generation
Here we ask throught API to Google Gemini to derive N prompts for each class. We define a clear prompt for Gemini where we define the number of prompts for each class and define a structured form of the output to better menage it.

In [33]:
NUM_OF_VARIANTS = 10

SYSTEM_PROMPT = f"""You are a vision-language expert.

Input format
• The user will give one Oxford-102 flower class name (e.g. “Daffodil”).

Output format
• Return **exactly {NUM_OF_VARIANTS}** short, visually grounded prompts.
• Separate the prompts with **“; ”** (semicolon + space).
• Do **not** add a semicolon after the final prompt.
• Output nothing else.

Content rules for each prompt
1. Start with a different prefix chosen from this set (use each at most once):
   “A photo of a”, “An image of a”, “A close-up of a”, “A macro shot of a”,
   “A studio photo of a”, “An outdoor photo of a”, “A botanical plate of a”,
   “A well-lit photograph of a”, “A side view of a”, “A top-down view of a”.
2. Immediately include the **number of flowers** in the frame (e.g. “single”, “one”, “pair of”, “three”).
3. Name the flower class exactly as given.
4. Mention dominant colour(s) plus 1-2 key visual cues (petal count/shape, centre, leaves, stem).
5. Keep each prompt ≤ 25 words.
6. Use only information visible in a typical image; avoid subjective or non-visual terms (beautiful, symbolic, fragrant, etc.).
7. End each prompt **without** a period.

Return the ten prompts in one line, separated by “; ”, and nothing else.
"""

In [36]:
from google import genai
from google.genai import types

# GEMINI API KEY IN SECRET's SECTION (left menu)
''' 
# If noteboock in Google Colab
from google.colab import userdata
userdata.get('GEMINI_API_KEY')
'''

# If noteboock in Jupiter
from dotenv import load_dotenv

load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
# print(gemini_api_key)

client = genai.Client(api_key=GEMINI_API_KEY)

In [38]:
def generate_classes_prompt_variants(classes: list):
    '''
    This function generate a set of N prompts for each class in the input list parameter
    '''
    prompt_variants_dict = {}  

    # Iterate each class: i is the class number, c is the class label
    for i, c in enumerate(classes):
        try:
            # Send the request to gemini with API
            response = client.models.generate_content(
                model="gemini-2.5-flash",
                config=types.GenerateContentConfig(
                    system_instruction=SYSTEM_PROMPT,
                    temperature=0.1
                ),
                contents=c
            )
            # Extract the response
            all_prompts = response.text
            prompt_variants_dict[c] = all_prompts.split('; ')
            print(f"✅ Finished class {c} - {i+1}/{len(classes)}")

        except Exception as e:
            print(f"❌ Error generating class '{c}' - {i+1}/{len(classes)}: {e}")
            prompt_variants_dict[c] = []  # oppure `None`, a seconda di cosa preferisci

    return prompt_variants_dict

We initialize our system by define the prompts for each of the 102 classes of our database. Either this the code is orgnaized in order to flexible and easy to generate new prompts for new classes.

To avoid overload Gemnin and risk to get lower quality result we devide the classes in small batches of 6 classes each one.

In [57]:
def init_prompts():
    
    # orgnize classes in chunck of 6 units
    chunk_size = 6
    chunked_class_names = [CLASS_NAMES_12[i:i + chunk_size] for i in range(0, len(CLASS_NAMES_12), chunk_size)]

    dict_list = []  # list containing the generated dictionaries
    for chunk in tqdm(chunked_class_names, desc="Generating dictionaries"):
        variants_dict = generate_classes_prompt_variants(chunk)   # generate the prompts for the chunk's classes
        dict_list.append(variants_dict)

    # Unify all the dictionries of the cunks in only one dictionary
    unified_dict = defaultdict(list)
    for diz in dict_list:
        for key, val in diz.items():
            unified_dict[key].extend(val)
            
    prompt_of_variants = dict(unified_dict)
        
    with open('generated_prompts.json', 'w') as f:
        json.dump(prompt_of_variants, f, indent=4)

In [58]:
# init_prompts()  

Generating dictionaries:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Finished class pink primrose - 1/6
✅ Finished class hard-leaved pocket orchid - 2/6
✅ Finished class canterbury bells - 3/6
✅ Finished class sweet pea - 4/6
✅ Finished class english marigold - 5/6


Generating dictionaries:  50%|█████     | 1/2 [00:37<00:37, 37.94s/it]

✅ Finished class tiger lily - 6/6
✅ Finished class moon orchid - 1/6
✅ Finished class bird of paradise - 2/6
✅ Finished class monkshood - 3/6
✅ Finished class globe thistle - 4/6
✅ Finished class snapdragon - 5/6


Generating dictionaries: 100%|██████████| 2/2 [01:18<00:00, 39.31s/it]

✅ Finished class colt's foot - 6/6


In [44]:
def load_prompts(json_path='generated_prompts.json'):
    """
    Reads the generated prompts from a JSON file and returns them as a dictionary.
    
    Args:
        json_path (str): Path to the JSON file containing the prompts.
    
    Returns:
        dict: Dictionary with class names as keys and list of prompt variants as values.
    """
    with open(json_path, 'r') as f:
        prompt_dict = json.load(f)
    return prompt_dict

# TESTING
1) Vanilla CLIP Zero Shot Classification
2) Only Prmpts Solution

In [7]:
def split_data(dataset):
    # these two lists will store the sample indexes
    base_categories_samples = []
    novel_categories_samples = []

    # we know that the first 50 classes belong to the Base group, whilt the others to Novel
    base_set = set(range(0, 51))

    # here we iterate over sample labels and also get the correspondent sample index
    for sample_id, label in enumerate(dataset._labels):
        if label in base_set:
            base_categories_samples.append(sample_id)
        else:
            novel_categories_samples.append(sample_id)

    # here we create the dataset subsets; the torch Subset is just a wrapper around the dataset
    # it simply stores the subset indexes and the original dataset (your_subset.dataset)
    # when asking for sample i in the subset, torch will look for its original position in the dataset and retrieve it
    base_dataset = torch.utils.data.Subset(dataset, base_categories_samples)
    novel_dataset = torch.utils.data.Subset(dataset, novel_categories_samples)
    
    return base_dataset, novel_dataset

In [14]:
import torch, torchvision
from torchvision.datasets import Flowers102

def get_test_data(data_dir="./data", transform=None):
    """Load Flowers102 train, validation and test sets.
    Args:
        data_dir (str): Directory where the dataset will be stored.
        transform (torch.Compose)
    Returns:
        tuple: A tuple containing the train, validation, and test sets.
    """
    # Load Oxford Flowers test split and class names
    test = torchvision.datasets.Flowers102(root=data_dir, split="test", download=True, transform=transform)
    return test

## VANILLA CLIP ZERO SHOT CLASSIFICATION

In [40]:
@torch.no_grad() # we don't want gradients
def eval_vanilla_clip(model, dataset, categories, batch_size, device, label=""):
    # let's set the model in evaluation mode
    model.eval()

    # Remap labels into a contiguous set starting from zero
    contig_cat2idx = {cat: idx for idx, cat in enumerate(categories)}

    # here we apply the standard CLIP template used for oxford flowers to all categories
    # and immediately tokenize each sentence (convert natural language into numbers - feel free to print the text input to inspect them)
    text_inputs = clip.tokenize(
        [f"a photo of a {CLASS_NAMES[c]}, a type of flower." for c in categories]
    ).to(device)

    # we can encode the text features once as they are shared for all images
    # therefore we do it outside the evaluation loop
    text_features = model.encode_text(text_inputs)
    # and here we normalize them (standard pratice with CLIP)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # simple dataloader creation
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # here we store the number of correct predictions we will make
    correct_predictions = 0
    for image, target in tqdm(dataloader, desc=label):
        # base categories range from 0 to 50, whil novel ones from 51 to 101
        # therefore we must map categories to the [0, 50], otherwise we will have wrong predictions
        # Map targets in contiguous set starting from zero
        # Labels needs to be .long() in pytorch
        target = torch.Tensor([contig_cat2idx[t.item()] for t in target]).long()

        image = image.to(device)
        target = target.to(device)

        # forward image through CLIP image encoder
        image_features = model.encode_image(image)
        # and normalize
        image_features /= image_features.norm(dim=-1, keepdim=True)

        # here cosine similarity between image and text features and keep the argmax for every row (every image)
        predicted_class = (image_features @ text_features.T).argmax(dim=-1)
        # now we check which are correct, and sum them (False == 0, True == 1)
        correct_predictions += (predicted_class == target).sum().item()

    # and now we compute the accuracy
    accuracy = correct_predictions / len(dataset)
    return accuracy

In [42]:
import clip
import torch
from torchvision.datasets import Flowers102
from torchvision import transforms

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess_clip = clip.load("ViT-B/16", device=device)

test_dataset = get_test_data(transform=preprocess_clip)
base_test_dataset, novel_test_dataset = split_data(test_dataset)

base_classes = list(range(51))           # [0, 1, ..., 50]
novel_classes = list(range(51, 102))     # [51, 52, ..., 101]

base_accuracy = eval_vanilla_clip(model=model, dataset=base_test_dataset, categories=base_classes, batch_size=128, device=device, label="🧠 Zero-shot evaluation on Base Classes")
novel_accuracy = eval_vanilla_clip(model=model, dataset=novel_test_dataset, categories=novel_classes, batch_size=128, device=device, label="🧠 Zero-shot evaluation on Novel Classes")

print()
print(f"🔍 Base classes accuracy: {base_accuracy*100:.2f}%")
print(f"🔍 Novel classes accuracy: {novel_accuracy*100:.2f}%")

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
[51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101]


🧠 Zero-shot evaluation on Novel Classes: 100%|██████████| 29/29 [00:18<00:00,  1.57it/s]


🔍 Base classes accuracy: 71.33%
🔍 Novel classes accuracy: 78.24%


## ONLY PRMPTS AGUMENTATION

In [46]:
import torch

@torch.no_grad() # we don't want gradients
def eval_only_prompts(model, dataset, prompt_dict, batch_size, device, label=""):

    model.eval()        # let's set the model in evaluation mode

    # get features (average embedding for each class)
    class_features = []

    # Ordine fisso → ci servirà per mappare i target
    ordered_keys = list(prompt_dict.keys())

    for k in ordered_keys:
        prompts = prompt_dict[k]                     # lista di stringhe

        embeddings = []
        for prompt in prompts:
            tokens = clip.tokenize(prompt, truncate=True).to(device)  # [N_prompts, 77]

            # Encode & normalize i prompt
            emb = model.encode_text(tokens)        # [N_prompts, 512]
            emb = emb / emb.norm(dim=-1, keepdim=True)
            embeddings.append(emb.squeeze(0))  # [512]

        # Media + rinormalizzazione
        cls_emb = torch.stack(embeddings).mean(dim=0)                   # [512]
        cls_emb = cls_emb / cls_emb.norm()

        class_features.append(cls_emb)

    # Stack → [num_classi, 512]
    text_features = torch.stack(class_features)


    # Remap labels into a contiguous set starting from zero
    contig_cat2idx = {idx: idx for idx, cat in enumerate(ordered_keys)}


    # simple dataloader creation
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # here we store the number of correct predictions we will make
    correct_predictions = 0
    for image, target in tqdm(dataloader, desc=label):
        # base categories range from 0 to 50, whil novel ones from 51 to 101
        # therefore we must map categories to the [0, 50], otherwise we will have wrong predictions
        # Map targets in contiguous set starting from zero
        # Labels needs to be .long() in pytorch
        target = torch.Tensor([contig_cat2idx[t.item()] for t in target]).long()

        image = image.to(device)
        target = target.to(device)

        # forward image through CLIP image encoder
        image_features = model.encode_image(image)
        # and normalize
        image_features /= image_features.norm(dim=-1, keepdim=True)

        # here cosine similarity between image and text features and keep the argmax for every row (every image)
        predicted_class = (image_features @ text_features.T).argmax(dim=-1)
        # now we check which are correct, and sum them (False == 0, True == 1)
        correct_predictions += (predicted_class == target).sum().item()


    # and now we compute the accuracy
    accuracy = correct_predictions / len(dataset)
    return accuracy

In [47]:
import clip
import torch
from torchvision.datasets import Flowers102
from torchvision import transforms

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess_clip = clip.load("ViT-B/16", device=device)

test_dataset = get_test_data(transform=preprocess_clip)
base_test_dataset, novel_test_dataset = split_data(test_dataset)

generated_prompts = load_prompts()

base_accuracy = eval_only_prompts(model=model, dataset=base_test_dataset, prompt_dict=generated_prompts, batch_size=128, device=device, label="🧠 Zero-shot evaluation on Base Classes")
novel_accuracy = eval_only_prompts(model=model, dataset=novel_test_dataset, prompt_dict=generated_prompts, batch_size=128, device=device, label="🧠 Zero-shot evaluation on Novel Classes")

print()
print(f"🔍 Base classes accuracy: {base_accuracy*100:.2f}%")
print(f"🔍 Novel classes accuracy: {novel_accuracy*100:.2f}%")

🧠 Zero-shot evaluation on Novel Classes: 100%|██████████| 29/29 [00:19<00:00,  1.47it/s]


🔍 Base classes accuracy: 66.96%
🔍 Novel classes accuracy: 70.08%
